In [6]:
# Импорт необходимых библиотек
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn import metrics
import xgboost as xgb
from joblib import dump, load

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

#### 1. Определение метрики

In [7]:
# Определение метрики (нормализованный коэффициент Джини)
# Для оценки будем использовать нормализованный коэффициент Джини
# эта метрика позволяет минимизировать ошибки 2 рода - отказ в кредите ответственному заемщику 
# то есть глобально, она направлена на увеличение потенциальной прибыли от выданных кредитов

def norm_gini(actual, predict):
    return 2*metrics.roc_auc_score(actual, predict)-1

#### 2. Загрузка и предварительная обработка данных

In [8]:
logger.info('Загрузка данных')
credits = pd.read_excel('credits.xlsx', header=1, index_col=0)
logger.info('Готово')
#print(credits.head())
#print(credits.info())
# нулевых данных нет, предобработка не нужна

INFO:__main__:Загрузка данных
INFO:__main__:Готово


#### 3. Разделение данных на обучающую и тестовую выборки

In [9]:
# разделим датасет на обучающую и тестовую выборки
train = credits[credits.is_test == 0]
test = credits[credits.is_test == 1]

#### 4. Подготовка данных для обучения и тестирования модели

In [10]:
# уберём лишние столбцы, нам нужны только столбцы с первого по 'res_npl_15'
logger.info('Предобработка данных')
y_col = 'res_npl_15'
train = train.loc[:, :y_col]
train_y = train[y_col]
train_X = train.drop(columns=y_col)

test = test.loc[:, :y_col]
test_X = test.drop(columns=y_col)

#print(X_train, y_train)
logger.info('Готово')

INFO:__main__:Предобработка данных
INFO:__main__:Готово


#### 5. Подготовка метрики для оценки модели

In [11]:
logger.info('Готовим метрику')
# Создадим объект gini_scorer с использованием функции make_scorer
gini_scorer = metrics.make_scorer(norm_gini, greater_is_better = True)
logger.info('Готово')

INFO:__main__:Готовим метрику
INFO:__main__:Готово


#### 6. Настройка и обучение модели XGBoost

In [12]:
# Создадим экземпляр модели
xgb_model = xgb.XGBClassifier(use_label_encoder=False)

logger.info('Подбираем лучшие параметры модели')
parameters = { 'objective':['binary:logistic'],
              'learning_rate': [0.05],
              'max_depth': [4,5],
              'min_child_weight': [11],
              'eval_metric': ['logloss'],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [100,200],
              'seed': [42]}

# Обучим модель с использованием кросс-валидации для определения лучших параметров
clf = GridSearchCV(xgb_model, parameters, n_jobs=5,                     
                   scoring=gini_scorer,
                   verbose=10, refit=True)
logger.info('Готово')

INFO:__main__:Подбираем лучшие параметры модели
INFO:__main__:Готово


#### 7. Подготовка предсказаний и сохранение результатов

In [13]:
# Обучение модели и поиск по сетке
clf.fit(train_X, train_y)
print('Лучшие параметры: ', clf.best_params_)

# Сохраняем обученную модель
logger.info('Сохраняем саму модель')
dump(clf.best_estimator_, 'best_xgb_model.joblib')
logger.info('Модель сохранена как best_xgb_model.joblib')

# Предсказание результатов для тестовой выборки
logger.info('Готовим предсказания')
test_probs = clf.predict(test_X)
test_X[y_col] = test_probs
logger.info('Готово')

# Сохраним предсказания в файл Excel
logger.info('Выгрузка результата')
test_X.to_excel('Submission.xlsx')
logger.info('Готово')

Fitting 5 folds for each of 4 candidates, totalling 20 fits


INFO:__main__:Сохраняем саму модель
INFO:__main__:Модель сохранена как best_xgb_model.joblib
INFO:__main__:Готовим предсказания
INFO:__main__:Готово
INFO:__main__:Выгрузка результата


Лучшие параметры:  {'colsample_bytree': 0.7, 'eval_metric': 'logloss', 'learning_rate': 0.05, 'max_depth': 4, 'min_child_weight': 11, 'n_estimators': 200, 'objective': 'binary:logistic', 'seed': 42, 'subsample': 0.8}


INFO:__main__:Готово
